In [10]:
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.preprocessing.image import array_to_img
import os, os.path
import numpy as np
from keras.datasets import cifar10
from keras.models import Sequential
from keras.layers import Dense, Flatten, Conv2D, MaxPooling2D
from keras.losses import sparse_categorical_crossentropy
from keras.optimizers import Adam
from sklearn.model_selection import KFold
import cv2
from keras import backend as K

In [2]:
# Model configuration
batch_size = 16
img_width, img_height, img_num_channels = 32, 32, 3
loss_function = sparse_categorical_crossentropy
no_classes = 100
no_epochs = 50
optimizer = Adam()
verbosity = 1
num_folds = 10

In [3]:
# Determine shape of the data
input_shape = (img_width, img_height, img_num_channels)
path = 'C:\\Users\\Mohamed\\PycharmProjects\\pythonProject1\\Graduation\\CelebA_Spoof\\Data\\train\\live\\'
ImagesPaths = [x for x in os.listdir(path)]
imagesArray = np.zeros((1052,img_width,img_height,3),dtype = 'uint8')
imagesLabels = np.zeros((1052,1),dtype = 'uint8')


In [4]:
count = 0
for i in ImagesPaths:
    img = cv2.imread(path+i)
    img = cv2.resize(img,(img_width,img_height))
    # convert to numpy array
    imagesArray[count]= img
    imagesLabels[count]=np.array([0])
    count+=1

In [5]:
path = 'C:\\Users\\Mohamed\\PycharmProjects\\pythonProject1\\Graduation\\CelebA_Spoof\\Data\\train\\spoof\\'
ImagesPaths = [x for x in os.listdir(path)]

In [6]:
for i in ImagesPaths:
    img = cv2.imread(path+i)
    img = cv2.resize(img,(img_width,img_height))
    # convert to numpy array
    imagesArray[count]= img
    imagesLabels[count]=np.array([1])
    count+=1

In [7]:
imagesArray = np.asarray(imagesArray).astype(np.float32)
imagesLabels=np.asarray(imagesLabels).astype(np.float32)
#imagesLabels = np.array(imagesLabels,dtype = object)
imagesArray = imagesArray / 255
# Define per-fold score containers
acc_per_fold = []
loss_per_fold = []
kfold = KFold(n_splits=num_folds, shuffle=True)


In [12]:
# K-fold Cross Validation model evaluation
fold_no = 1
for train, test in kfold.split(imagesArray,imagesLabels):

    # Define the model architecture
    model = Sequential()
    model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=input_shape))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Flatten())
    model.add(Dense(256, activation='relu'))
    model.add(Dense(128, activation='relu'))
    model.add(Dense(no_classes, activation='softmax'))

    def recall_m(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

    def precision_m(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision

    def f1_m(y_true, y_pred):
        precision = precision_m(y_true, y_pred)
        recall = recall_m(y_true, y_pred)
        return 2*((precision*recall)/(precision+recall+K.epsilon()))

    # Compile the model
    model.compile(loss=loss_function,
                optimizer=optimizer,
                metrics=['accuracy',f1_m,precision_m, recall_m])


    # Generate a print
    print('------------------------------------------------------------------------')
    print(f'Training for fold {fold_no} ...')

    # Fit data to model
    history = model.fit(imagesArray[train], imagesLabels[train],
              batch_size=batch_size,
              epochs=no_epochs,
              verbose=verbosity)

    # Generate generalization metrics
    scores = model.evaluate(imagesArray[test], imagesLabels[test], verbose=0)
    print(f'Score for fold {fold_no}: {model.metrics_names[0]} of {scores[0]}; {model.metrics_names[1]} of {scores[1]*100}%')
    acc_per_fold.append(scores[1] * 100)
    loss_per_fold.append(scores[0])

    # Increase fold number
    fold_no = fold_no + 1

------------------------------------------------------------------------
Training for fold 1 ...
Epoch 1/50
60/60 [==============================] - 1s 17ms/step - loss: 1.2826 - accuracy: 0.5486 - f1_m: 0.4549 - precision_m: 0.3667 - recall_m: 0.6486
Epoch 2/50
60/60 [==============================] - 1s 17ms/step - loss: 0.4549 - accuracy: 0.8118 - f1_m: 0.6601 - precision_m: 0.5021 - recall_m: 0.9963
Epoch 3/50
60/60 [==============================] - 1s 17ms/step - loss: 0.3150 - accuracy: 0.8763 - f1_m: 0.6662 - precision_m: 0.5104 - recall_m: 1.0000
Epoch 4/50
60/60 [==============================] - 1s 17ms/step - loss: 0.2310 - accuracy: 0.9049 - f1_m: 0.6613 - precision_m: 0.5031 - recall_m: 1.0000
Epoch 5/50
60/60 [==============================] - 1s 18ms/step - loss: 0.2027 - accuracy: 0.9144 - f1_m: 0.6603 - precision_m: 0.5031 - recall_m: 1.0000 0s - loss: 0.2110 - accuracy: 0.9125 - f1_m: 0.6139 - prec
Epoch 6/50
60/60 [==============================] - 1s 18ms/step - lo

60/60 [==============================] - 1s 16ms/step - loss: 0.8164 - accuracy: 0.6543 - f1_m: 0.5818 - precision_m: 0.4480 - recall_m: 0.8813
Epoch 2/50
60/60 [==============================] - 1s 16ms/step - loss: 0.3305 - accuracy: 0.8319 - f1_m: 0.6485 - precision_m: 0.4936 - recall_m: 0.9812
Epoch 3/50
60/60 [==============================] - 1s 17ms/step - loss: 0.2173 - accuracy: 0.9133 - f1_m: 0.6598 - precision_m: 0.5010 - recall_m: 1.0000
Epoch 4/50
60/60 [==============================] - 1s 17ms/step - loss: 0.2042 - accuracy: 0.9154 - f1_m: 0.6568 - precision_m: 0.5010 - recall_m: 1.0000
Epoch 5/50
60/60 [==============================] - 1s 17ms/step - loss: 0.1340 - accuracy: 0.9493 - f1_m: 0.6612 - precision_m: 0.5083 - recall_m: 1.0000
Epoch 6/50
60/60 [==============================] - 1s 17ms/step - loss: 0.1476 - accuracy: 0.9419 - f1_m: 0.6479 - precision_m: 0.4938 - recall_m: 0.9833
Epoch 7/50
60/60 [==============================] - 1s 17ms/step - loss: 0.1463 -

60/60 [==============================] - 1s 17ms/step - loss: 0.7326 - accuracy: 0.6494 - f1_m: 0.6213 - precision_m: 0.4779 - recall_m: 0.9192
Epoch 2/50
60/60 [==============================] - 1s 17ms/step - loss: 0.3044 - accuracy: 0.8596 - f1_m: 0.6570 - precision_m: 0.5020 - recall_m: 0.9860
Epoch 3/50
60/60 [==============================] - 1s 17ms/step - loss: 0.2612 - accuracy: 0.8775 - f1_m: 0.6520 - precision_m: 0.4975 - recall_m: 0.9880
Epoch 4/50
60/60 [==============================] - 1s 17ms/step - loss: 0.1917 - accuracy: 0.9250 - f1_m: 0.6620 - precision_m: 0.5038 - recall_m: 1.0000
Epoch 5/50
60/60 [==============================] - 1s 17ms/step - loss: 0.1564 - accuracy: 0.9293 - f1_m: 0.6587 - precision_m: 0.4993 - recall_m: 1.0000
Epoch 6/50
60/60 [==============================] - 1s 17ms/step - loss: 0.1239 - accuracy: 0.9493 - f1_m: 0.6490 - precision_m: 0.4948 - recall_m: 0.9833
Epoch 7/50
60/60 [==============================] - 1s 17ms/step - loss: 0.0956 -

60/60 [==============================] - 1s 20ms/step - loss: 1.3285 - accuracy: 0.5671 - f1_m: 0.5507 - precision_m: 0.4280 - recall_m: 0.8225- ETA: 0s - loss: 2.4728 - accuracy: 0.5028 - f1_m: 0.3808 - precision_m:
Epoch 2/50
60/60 [==============================] - 1s 21ms/step - loss: 0.4384 - accuracy: 0.7941 - f1_m: 0.6582 - precision_m: 0.5017 - recall_m: 1.0000 0s - loss: 0.4876 - accuracy: 0.7589 - f1_m: 0.6808 - precision_m: 0
Epoch 3/50
60/60 [==============================] - 1s 24ms/step - loss: 0.3245 - accuracy: 0.8638 - f1_m: 0.6554 - precision_m: 0.4972 - recall_m: 1.0000
Epoch 4/50
60/60 [==============================] - 1s 24ms/step - loss: 0.2876 - accuracy: 0.8828 - f1_m: 0.6601 - precision_m: 0.5017 - recall_m: 1.0000
Epoch 5/50
60/60 [==============================] - 1s 22ms/step - loss: 0.3010 - accuracy: 0.8828 - f1_m: 0.6546 - precision_m: 0.4978 - recall_m: 1.0000 0s - loss: 0.3915 - accuracy: 0.8333 - f1_m: 0.6397 - pr
Epoch 6/50
60/60 [===================

Epoch 1/50
60/60 [==============================] - 1s 17ms/step - loss: 0.9674 - accuracy: 0.6230 - f1_m: 0.5907 - precision_m: 0.4600 - recall_m: 0.8733
Epoch 2/50
60/60 [==============================] - 1s 18ms/step - loss: 0.3022 - accuracy: 0.8691 - f1_m: 0.6496 - precision_m: 0.4941 - recall_m: 1.0000
Epoch 3/50
60/60 [==============================] - 1s 17ms/step - loss: 0.2501 - accuracy: 0.9007 - f1_m: 0.6522 - precision_m: 0.4941 - recall_m: 1.0000
Epoch 4/50
60/60 [==============================] - 1s 17ms/step - loss: 0.2027 - accuracy: 0.9145 - f1_m: 0.6446 - precision_m: 0.4896 - recall_m: 0.9833
Epoch 5/50
60/60 [==============================] - 1s 18ms/step - loss: 0.1721 - accuracy: 0.9293 - f1_m: 0.6446 - precision_m: 0.4896 - recall_m: 0.9833
Epoch 6/50
60/60 [==============================] - 1s 17ms/step - loss: 0.1411 - accuracy: 0.9377 - f1_m: 0.6521 - precision_m: 0.4941 - recall_m: 1.0000
Epoch 7/50
60/60 [==============================] - 1s 19ms/step - los

60/60 [==============================] - 1s 17ms/step - loss: 0.7798 - accuracy: 0.6251 - f1_m: 0.6097 - precision_m: 0.4652 - recall_m: 0.9368
Epoch 2/50
60/60 [==============================] - 1s 17ms/step - loss: 0.3088 - accuracy: 0.8522 - f1_m: 0.6365 - precision_m: 0.4858 - recall_m: 0.9790
Epoch 3/50
60/60 [==============================] - 1s 17ms/step - loss: 0.2275 - accuracy: 0.9124 - f1_m: 0.6569 - precision_m: 0.5000 - recall_m: 1.0000
Epoch 4/50
60/60 [==============================] - 1s 17ms/step - loss: 0.2108 - accuracy: 0.9166 - f1_m: 0.6552 - precision_m: 0.4955 - recall_m: 1.0000
Epoch 5/50
60/60 [==============================] - 1s 17ms/step - loss: 0.1717 - accuracy: 0.9219 - f1_m: 0.6572 - precision_m: 0.5000 - recall_m: 1.0000
Epoch 6/50
60/60 [==============================] - 1s 18ms/step - loss: 0.1486 - accuracy: 0.9398 - f1_m: 0.6551 - precision_m: 0.5000 - recall_m: 1.0000
Epoch 7/50
60/60 [==============================] - 1s 17ms/step - loss: 0.1141 -

60/60 [==============================] - 1s 17ms/step - loss: 0.8502 - accuracy: 0.6494 - f1_m: 0.6068 - precision_m: 0.4716 - recall_m: 0.9000
Epoch 2/50
60/60 [==============================] - 1s 17ms/step - loss: 0.2722 - accuracy: 0.8765 - f1_m: 0.6559 - precision_m: 0.4999 - recall_m: 1.0000
Epoch 3/50
60/60 [==============================] - 1s 17ms/step - loss: 0.2224 - accuracy: 0.9018 - f1_m: 0.6651 - precision_m: 0.5077 - recall_m: 1.0000
Epoch 4/50
60/60 [==============================] - 1s 17ms/step - loss: 0.1805 - accuracy: 0.9187 - f1_m: 0.6599 - precision_m: 0.5028 - recall_m: 1.0000
Epoch 5/50
60/60 [==============================] - 1s 17ms/step - loss: 0.1324 - accuracy: 0.9483 - f1_m: 0.6581 - precision_m: 0.4983 - recall_m: 1.0000
Epoch 6/50
60/60 [==============================] - 1s 17ms/step - loss: 0.1143 - accuracy: 0.9493 - f1_m: 0.6569 - precision_m: 0.4983 - recall_m: 1.0000
Epoch 7/50
60/60 [==============================] - 1s 18ms/step - loss: 0.0836 -

60/60 [==============================] - 1s 17ms/step - loss: 1.0321 - accuracy: 0.5945 - f1_m: 0.5927 - precision_m: 0.4594 - recall_m: 0.8889A: 0s - loss: 1.4602 - accuracy: 0.4961 - f1_m: 0.5308 - precision_m: 0.4167 
Epoch 2/50
60/60 [==============================] - 1s 17ms/step - loss: 0.3432 - accuracy: 0.8490 - f1_m: 0.6489 - precision_m: 0.4958 - recall_m: 0.9833
Epoch 3/50
60/60 [==============================] - 1s 17ms/step - loss: 0.2598 - accuracy: 0.8923 - f1_m: 0.6570 - precision_m: 0.5003 - recall_m: 1.0000
Epoch 4/50
60/60 [==============================] - 1s 17ms/step - loss: 0.2101 - accuracy: 0.9145 - f1_m: 0.6491 - precision_m: 0.4958 - recall_m: 0.9833
Epoch 5/50
60/60 [==============================] - 1s 17ms/step - loss: 0.1934 - accuracy: 0.9197 - f1_m: 0.6576 - precision_m: 0.5003 - recall_m: 1.0000
Epoch 6/50
60/60 [==============================] - 1s 17ms/step - loss: 0.1668 - accuracy: 0.9303 - f1_m: 0.6590 - precision_m: 0.5003 - recall_m: 1.0000
Epoc

60/60 [==============================] - 1s 17ms/step - loss: 0.8104 - accuracy: 0.7033 - f1_m: 0.5901 - precision_m: 0.4533 - recall_m: 0.8976
Epoch 2/50
60/60 [==============================] - 1s 17ms/step - loss: 0.3144 - accuracy: 0.8733 - f1_m: 0.6486 - precision_m: 0.4944 - recall_m: 1.0000
Epoch 3/50
60/60 [==============================] - 1s 17ms/step - loss: 0.2414 - accuracy: 0.9018 - f1_m: 0.6529 - precision_m: 0.4953 - recall_m: 1.0000
Epoch 4/50
60/60 [==============================] - 1s 17ms/step - loss: 0.2202 - accuracy: 0.9166 - f1_m: 0.6517 - precision_m: 0.4944 - recall_m: 1.0000
Epoch 5/50
60/60 [==============================] - 1s 17ms/step - loss: 0.1549 - accuracy: 0.9409 - f1_m: 0.6398 - precision_m: 0.4854 - recall_m: 0.9833
Epoch 6/50
60/60 [==============================] - 1s 17ms/step - loss: 0.1332 - accuracy: 0.9483 - f1_m: 0.6500 - precision_m: 0.4905 - recall_m: 1.0000
Epoch 7/50
60/60 [==============================] - 1s 17ms/step - loss: 0.1023 -

60/60 [==============================] - 1s 21ms/step - loss: 0.9304 - accuracy: 0.4995 - f1_m: 0.6021 - precision_m: 0.4716 - recall_m: 0.8817
Epoch 2/50
60/60 [==============================] - 1s 21ms/step - loss: 0.5788 - accuracy: 0.6779 - f1_m: 0.6683 - precision_m: 0.5112 - recall_m: 0.9979
Epoch 3/50
60/60 [==============================] - 1s 21ms/step - loss: 0.3152 - accuracy: 0.8532 - f1_m: 0.6667 - precision_m: 0.5082 - recall_m: 1.0000
Epoch 4/50
60/60 [==============================] - 1s 20ms/step - loss: 0.2499 - accuracy: 0.8902 - f1_m: 0.6634 - precision_m: 0.5076 - recall_m: 1.0000
Epoch 5/50
60/60 [==============================] - 1s 17ms/step - loss: 0.2006 - accuracy: 0.9250 - f1_m: 0.6617 - precision_m: 0.5076 - recall_m: 1.0000
Epoch 6/50
60/60 [==============================] - 1s 17ms/step - loss: 0.2038 - accuracy: 0.9166 - f1_m: 0.6661 - precision_m: 0.5076 - recall_m: 1.0000
Epoch 7/50
60/60 [==============================] - 1s 17ms/step - loss: 0.1721 -

In [13]:
# == Provide average scores ==
print('------------------------------------------------------------------------')
print('Score per fold')
for i in range(0, len(acc_per_fold)):
  print('------------------------------------------------------------------------')
  print(f'> Fold {i+1} - Loss: {loss_per_fold[i]} - Accuracy: {acc_per_fold[i]}%')
print('------------------------------------------------------------------------')
print('Average scores for all folds:')
print(f'> Accuracy: {np.mean(acc_per_fold)} (+- {np.std(acc_per_fold)})')
print(f'> Loss: {np.mean(loss_per_fold)}')
print('------------------------------------------------------------------------')

------------------------------------------------------------------------
Score per fold
------------------------------------------------------------------------
> Fold 1 - Loss: 0.09042751789093018 - Accuracy: 98.11320900917053%
------------------------------------------------------------------------
> Fold 2 - Loss: 0.7726038098335266 - Accuracy: 94.33962106704712%
------------------------------------------------------------------------
> Fold 3 - Loss: 0.5121167302131653 - Accuracy: 96.1904764175415%
------------------------------------------------------------------------
> Fold 4 - Loss: 0.03362816572189331 - Accuracy: 98.09523820877075%
------------------------------------------------------------------------
> Fold 5 - Loss: 0.576591432094574 - Accuracy: 97.14285731315613%
------------------------------------------------------------------------
> Fold 6 - Loss: 0.13382640480995178 - Accuracy: 98.09523820877075%
-----------------------------------------------------------------------

In [14]:
model.evaluate(imagesArray[test], imagesLabels[test], verbose=0)

[0.5738533139228821, 0.9333333373069763, 0.529411792755127, 0.515625, 0.75]